# Import library

In [82]:
# #importing libraries
# import csv
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.impute import KNNImputer
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer
# from sklearn.ensemble import RandomForestClassifier

# from boruta import BorutaPy
# from BorutaShap import BorutaShap
# from sklearn.feature_selection import RFE

# from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.combine import SMOTEENN

# from collections import Counter
# from sklearn.svm import SVC

# from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, precision_score, recall_score, roc_auc_score

In [83]:
# from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, precision_score, recall_score, roc_auc_score

# Read df

In [84]:
#this function is to read, transform and join 2 data frame

def read_features():
    path = 'input/secom.data'
    df = pd.read_csv(path, delimiter=' ', header=None, na_values=['NaN'])
    df.columns = ['feature_'+str(x+1) for x in range(len(df.columns))]
    return df



def read_target():
    path = 'input/secom_labels.data'
    df = pd.read_csv(path, delimiter=' ', header=None, na_values=['NaN'])
    df.columns = ['status','timestamp']
    df['timestamp'] = pd.to_datetime(df['timestamp'],dayfirst=True)
    return df

#for the testing purporse, trim to remain first 100 rows only
X = read_features()
y = read_target().iloc[:,0]

# Remove duplicated columns

In [85]:
#find the duplicated features (columns)
def remove_duplicated_columns(df):
    dict_duplicate_pair = {}
    dict_duplicate_matches = {}
    list_duplicate = []
    to_remove = []
    for i in range(0, len(df.columns)):
        l = []
        for j in range(i+1,len(df.columns)):
            dict_duplicate_pair[str(i+1)+';'+str(j+1)] = df.iloc[:,i].equals(df.iloc[:,j])
            if df.iloc[:,i].equals(df.iloc[:,j]) == True:
                if j not in list_duplicate:
                    l.append(j)
                    to_remove.append('feature_'+str(j+1))
                list_duplicate.append(i)
                list_duplicate.append(j)
        if len(l)!=0:
            dict_duplicate_matches[i] = l


    df_duplicate_pair = pd.DataFrame.from_dict(dict_duplicate_pair, orient='index')
    df_duplicate_pair.columns=['duplicate']

    df_duplicate_matches = pd.DataFrame.from_dict(dict_duplicate_matches, orient='index')

    
    df = df.drop(columns=to_remove, axis = 1)

    return df

# X = remove_duplicated_columns(X)
# X.shape


# Remove columns with Constant volatility (std=0)

In [86]:
def remove_constant_volatility(df):
    df_EDA= df.describe().T
    df_EDA= df_EDA[df_EDA["std"] == 0]
    df = df.drop(axis=1, columns=df_EDA.index)
    return df

# X = remove_constant_volatility(X)
# X.shape

# Remove columns with high %Missing values

In [87]:
def remove_cols_with_high_pct_null(df, null_threshold):
    list_column_with_pct_null = pd.concat([df.isnull().sum(), df.isnull().sum()/df.shape[0]],axis=1).rename(columns={0:'Missing_Records', 1:'Percentage (%)'})
    list_column_with_pct_null= list_column_with_pct_null[list_column_with_pct_null["Percentage (%)"] >= null_threshold]
    df = df.drop(axis=1, columns=list_column_with_pct_null.index)
    return df

# X = remove_cols_with_high_pct_null(X, 0.8)
# X.shape

# Split data

In [88]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1,stratify=y)

# Outlier treatment

In [89]:
#how = ['NaN', '3s']
def replace_outlier(df, how):
    for col in df:
        ll_col = df[col].mean() - 3 * df[col].std()
        ul_col = df[col].mean() + 3 * df[col].std()
        if how == 'NaN':
            df[col] = np.where(df[col]>ul_col,np.NaN,np.where(df[col]<ll_col,np.NaN,df[col]))
        elif how == '3s':
            df[col] = np.where(df[col]>ul_col,ul_col,np.where(df[col]<ll_col,ll_col,df[col]))
    return df

# Missing value Imputation

In [90]:
#which_weights = ['distance','uniform']

def impute_null_with_knn(X_train, X_test, which_weights):
    #First scale the data 
    scaler = MinMaxScaler()
    X_train = pd.DataFrame(scaler.fit_transform(X_train), columns= X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns= X_test.columns)

    knn = KNNImputer(n_neighbors=5, weights=which_weights) #check this neighbors = 5

    X_train = pd.DataFrame(knn.fit_transform(X_train), columns=X_train.columns)
    X_test = pd.DataFrame(knn.transform(X_test), columns=X_test.columns)
    
    X_train = pd.DataFrame(scaler.inverse_transform(X_train), columns= X_train.columns)
    X_test = pd.DataFrame(scaler.inverse_transform(X_test), columns= X_test.columns)
    return X_train, X_test

#X_train = impute_null_with_knn(X_train)

In [91]:
def impute_null_with_mice(X_train, X_test): 
    imp = IterativeImputer(max_iter=5, verbose=0, imputation_order='roman', random_state=0)
    X_train = pd.DataFrame(imp.fit_transform(X_train), columns=X_train.columns)
    X_test = pd.DataFrame(imp.transform(X_test), columns=X_test.columns)
    return X_train, X_test

# Feature Selection

In [92]:
#list_method=['shap','gini']

def BorutaShap_FS (X, y, method_option) :
    modelshap = RandomForestClassifier(n_jobs=-1,n_estimators=100, max_depth=5, random_state=100)
    # define model for resp. classifier
    modelshap.fit(X,y)
    ##-- feature_names = np.array(X.columns)
    # define Boruta Sahp feature selection method
    feature_selector = BorutaShap(model=modelshap,
                              importance_measure=method_option,
                              classification=True)  # find all relevant features
    feature_selector.fit(X,y,n_trials=100, sample=False, verbose=False, random_state=100)  
    ##-- feature_selector.plot(which_features='accepted',figsize=(20,10))
    # call transform() on X to filter it down to selected features
    return  feature_selector.Subset()

In [93]:
#RFE function with random forest

def RFE_FS (X, y, classify) :
    feature_names = np.array(X.columns)
    if classify == 'RF':
    # define random forest classifier
        model = RandomForestClassifier(n_jobs=-1, class_weight='balanced_subsample', max_depth=5, random_state=100)
        #model.fit(X, y)
        #rfe = RFE(estimator = model,n_features_to_select = 15)
    if classify== 'SVM':
        model = SVC(kernel='linear',C=5)
        #model.fit(X, y)
        #rfe = RFECV(estimator = model,scoring='accuracy')
    # find all relevant features
    model.fit(X, y)
    rfe = RFE(estimator = model,n_features_to_select = 20) #change to 20
    rfe.fit(X,y)

    # check selected features
    ##-- rfe.support_

    # check ranking of features
    ##--rfe.ranking_

    # zip feature names, ranks, and decisions 
    # feature_ranks = list(zip(feature_names, 
    #                          rfe.ranking_, 
    #                          rfe.support_))

    # print the results
    ##--for feat in feature_ranks:
    ##--    print('Feature: {:<30} Rank: {},  Keep: {}'.format(feat[0], feat[1], feat[2]))
        
    final_features_rfe = list()
    indexes = np.where(rfe.ranking_ <= 2) #change to 2
    for x in np.nditer(indexes):
        final_features_rfe.append(feature_names[x])
    ##-- print(final_features_rfe)
    
 # call transform() on X to filter it down to selected features
    return pd.DataFrame(X.filter(final_features_rfe))

In [94]:
#Boruta function with random forest

def BorutaPy_FS (X, y) :
    feature_names = np.array(X.columns)

    # define random forest classifier
    model = RandomForestClassifier(n_jobs=-1, class_weight='balanced_subsample', max_depth=5, random_state=100)
    model.fit(X, y)
    # define Boruta feature selection method
    
    feature_selector = BorutaPy(model, n_estimators='auto', verbose=0, random_state=100, max_iter=140)

    # find all relevant features
    feature_selector.fit(X.to_numpy(),y)

    # check selected features
    ##--feature_selector.support_

    # check ranking of features
    ##--feature_ranking=feature_selector.ranking_

    # zip feature names, ranks, and decisions 
    # feature_ranks = list(zip(feature_names, 
    #                          feature_selector.ranking_, 
    #                          feature_selector.support_))

    # print the results
    ##--for feat in feature_ranks:
    ##--    print('Feature: {:<30} Rank: {},  Keep: {}'.format(feat[0], feat[1], feat[2]))
        
    final_features = list()
    indexes = np.where(feature_selector.ranking_ <= 2) #change to 2
    for x in np.nditer(indexes):
        final_features.append(feature_names[x])
    ##--print(final_features)
    
 # call transform() on X to filter it down to selected features
    return pd.DataFrame(X.filter(final_features))

# Multicolinearity treatement

In [95]:
#Remove the highly collinear features from data
def remove_collinear_features(x, threshold):
    '''
    Objective:
        Remove collinear features in a dataframe with a correlation coefficient
        greater than the threshold. Removing collinear features can help a model 
        to generalize and improves the interpretability of the model.

    Inputs: 
        x: features dataframe
        threshold: features with correlations greater than this value are removed

    Output: 
        dataframe that contains only the non-highly-collinear features
    '''

    # Calculate the correlation matrix
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []

    # Iterate through the correlation matrix and compare correlations
    for i in iters:
        for j in range(i+1):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = abs(item.values)

            # If correlation exceeds the threshold
            if val >= threshold:
                #Print the correlated features and the correlation value
                #print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
                drop_cols.append(col.values[0])

    # Drop one of each pair of correlated columns
    drops = set(drop_cols)
    x = x.drop(columns=drops)

    return x

#remove_collinear_features(X, 0.7)

# Balancing

In [96]:
def sampling(X_train, y_train, sampler):
    
    #SMOTE
    if sampler == 'SMOTE':
        sampler = SMOTE(random_state=100)    
    
    #ROSE
    if sampler == 'ROSE':
        sampler = RandomOverSampler(random_state=100, shrinkage=1)

    #ADASYN
    if sampler == 'ADASYN':
        sampler = ADASYN(random_state=100)
    

    #SMOTTEENN
    if sampler == 'SMOTEENN' :
        sampler = SMOTEENN(random_state=100)
        
        
    #Random under Sampling
    if sampler == "randomunder":
        sampler = RandomUnderSampler(random_state=100)

    X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
    #counter = Counter(y_resampled)
    #print(counter)
    
    return X_resampled, y_resampled

# X_train, y_train = sampling(X_train, y_train,'SMOTE')
# X_train.shape

# Model

In [99]:
which_model = ['RF', 'LR']

def run_model(X_train, y_train, X_test, y_test, which_model):

    if which_model == 'RF':
    # building model before balancing data
        model = RandomForestClassifier(random_state=1, n_estimators=1000, max_depth=5)
    elif which_model == 'LR':
        model = LogisticRegression(random_state=1)
    
    model.fit(X_train,y_train)


    #For TEST SPLIT
    y_pred= model.predict(X_test)
    cf_matrix = confusion_matrix(y_test, y_pred)
    accuracy= accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred) ##
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    specificity = cf_matrix[1][1] / ( cf_matrix[1][1] + cf_matrix[1][0] )
    auc = roc_auc_score(y_test, y_pred)
    type_1_error_FP = cf_matrix[0][1]
    type_2_error_FN = cf_matrix[1][0]
    #Note by default 1 is the positive label. Therefore, -1 is negative
    #bad waffe -> 2 line of matrix -> POSITIVE -> data = -1

    #For TRAIN SPLIT
    y_pred_train= model.predict(X_train)
    cf_matrix_train = confusion_matrix(y_train, y_pred_train)
    accuracy_train= accuracy_score(y_train, y_pred_train)
    f1_train = f1_score(y_train, y_pred_train) ##
    precision_train = precision_score(y_train, y_pred_train)
    recall_train = recall_score(y_train, y_pred_train)
    specificity_train = cf_matrix_train[1][1] / ( cf_matrix_train[1][1] + cf_matrix_train[1][0] )
    auc_train = roc_auc_score(y_train, y_pred_train)
    type_1_error_FP_train = cf_matrix_train[0][1]
    type_2_error_FN_train = cf_matrix_train[1][0]


    return cf_matrix, accuracy, f1, precision, recall, specificity, type_1_error_FP, type_2_error_FN, auc, cf_matrix_train, accuracy_train, f1_train, precision_train, recall_train, specificity_train, type_1_error_FP_train, type_2_error_FN_train, auc_train

#run_model(X_train, y_train, X_test, y_test)

## Combine

In [ ]:
# fast_track_cols = X_test.columns
# fast_track_cols

In [80]:
X = read_features()
y = read_target().iloc[:,0]

result = []
i = 0

#f = open('output/tracker.csv', 'w')
# create the csv writer
#writer = csv.writer(f)



#step 1:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1, stratify=y)
#--------------> we should try differeny split

#-----------
#step 2:
X_train = remove_duplicated_columns(X_train)
#step 3:
X_train = remove_constant_volatility(X_train)
#step 4:
X_train = remove_cols_with_high_pct_null(X_train, 0.8) #this can be in the loop too, may be later

#step 5: remove the same columns from step 2-4 TRAIN_TEST split
X_test = X_test.loc[:,X_train.columns]


#------------


# X_train = X_train.loc[:, fast_track_cols]
# X_test = X_test.loc[:, fast_track_cols]

#------------


#step 6-9
replace_outlier_options = ['3s']
#replace_outlier_options = ['3s','NaN']
impute_null_options = ['knn__uniform','MICE']
#impute_null_options = ['knn__distance', 'MICE', 'knn__uniform']
FS_options = ['BoP','BoS__shap','RFE__RF']
#FS_options = ['BoP','BoS__shap', 'BoS__gini', 'RFE__RF', 'RFE__SVM' ]
sampling_options = ['ROSE']
#sampling_options = ['SMOTE','ROSE','ADASYN','SMOTEENN']
model_options = ['LR']
#model_options = ['LR', 'RF']

for replace_with in replace_outlier_options:
    for knn_weight in impute_null_options:
        for classifier_model in FS_options:
            for sampling_technique in sampling_options:
                for Model in model_options:
                    X_train_temp = X_train
                    X_test_temp = X_test
                    y_train_temp = y_train
                    y_test_temp = y_test

                    #step 6: oulier treatement (on both TRAIN & TEST split)
                    X_train_temp = replace_outlier(X_train_temp, replace_with)
                    X_test_temp = replace_outlier(X_test_temp, replace_with)
                    
                    #step 7: missing value imputation (on both TRAIN & TEST split)
                    if knn_weight == 'knn__distance' or knn_weight == 'knn__uniform':
                        X_train_temp, X_test_temp = impute_null_with_knn(X_train_temp, X_test_temp, knn_weight[-(len(knn_weight)-5):])
                    elif knn_weight == 'MICE':
                        X_train_temp, X_test_temp = impute_null_with_mice(X_train_temp, X_test_temp)

                    #step 8: feature selection (on both TRAIN & TEST split)
                    if classifier_model == 'BoS__shap' or classifier_model == 'BoS__gini':
                        X_train_temp = BorutaShap_FS(X_train_temp, y_train_temp, classifier_model[-(len(classifier_model)-5):])
                    elif classifier_model == 'RFE__RF' or classifier_model == 'RFE__SVM':
                        X_train_temp = RFE_FS(X_train_temp, y_train_temp, classifier_model[-(len(classifier_model)-5):])
                    elif classifier_model == 'BoP':
                        X_train_temp = BorutaPy_FS(X_train_temp, y_train_temp)
                    
                    #step 9: remove multilinear features
                    print('n_cols BEFORE multicolinearity treatement', X_train_temp.shape[1])
                    X_train_temp = remove_collinear_features(X_train_temp, 0.7)
                    print('n_cols AFTER multicolinearity treatement', X_train_temp.shape[1])

                    #apply the same result for TEST
                    X_test_temp = X_test_temp.loc[:,X_train_temp.columns]


                    #step 10: balancing only on TRAIN split
                    X_train_temp, y_train_temp = sampling(X_train_temp, y_train_temp, sampling_technique)


                    #step 11: train model, predict, and print scores
                    cf_matrix, accuracy, f1, precision, recall, specificity, type_1_error_FP, type_2_error_FN, auc, cf_matrix_train, accuracy_train, f1_train, precision_train, recall_train, specificity_train, type_1_error_FP_train, type_2_error_FN_train, auc_train = run_model(X_train_temp, y_train_temp, X_test_temp, y_test_temp, Model)

                    combined_technique = replace_with +'&'+ knn_weight +'&'+ classifier_model +'&'+ sampling_technique
                    result.append((i, combined_technique, X_train_temp.shape[1], replace_with, knn_weight, classifier_model, sampling_technique, X_train_temp.columns, cf_matrix, accuracy, f1, precision, recall, specificity, type_1_error_FP, type_2_error_FN, auc, cf_matrix_train, accuracy_train, f1_train, precision_train, recall_train, specificity_train, type_1_error_FP_train, type_2_error_FN_train, auc_train))
                    
                    print('attemp no.: ',i, combined_technique,' acc: ', accuracy,' accuracy_train: ',accuracy_train, ' f1: ', f1,' f1_train: ',f1_train, ' spec: ', specificity, ' :auc ', auc, ' cfm: ', '\n', cf_matrix, '\n cols', X_train_temp.shape[1], X_train_temp.columns, '\n')
                    #print(i, combined_technique)
                    #row = str(i) +' '+ combined_technique +' acc '+ str(accuracy) +' f1 '+ str(f1) +' acc_train '+ str(accuracy_train) +' f1_train '+ str(f1_train)
                    #writer.writerow(row)

                    i+=1
                    #print out datasets for backup
                    # X_train_temp.to_csv('output/X_train_temp_'+str(i)+combined_technique'.csv')
                    # X_test_temp.to_csv('output/X_test_temp_'+str(i)+combined_technique'.csv')
                    # y_train_temp.to_csv('output/y_train_temp_'+str(i)+combined_technique'.csv')
                    # y_test_temp.to_csv('output/y_test_temp_'+str(i)+combined_technique'.csv')



df_result = pd.DataFrame(result, columns = ['No.','combination','n_cols','outlier_replace_with', 'imputation_knn_weight', 'FS_classifier_model', 'balancing_sampling_technique', 'cols','cf_matrix', 'accuracy', 'f1', 'precision', 'recall', 'specificity', 'type_1_error_FP', 'type_2_error_FN', 'auc', 'cf_matrix_train', 'accuracy_train', 'f1_train', 'precision_train', 'recall_train', 'specificity_train', 'type_1_error_FP_train', 'type_2_error_FN_train', 'auc_train'])
df_result.to_csv('tracker/result.csv')
#print(result)
# close the file
#f.close()

n_cols BEFORE multicolinearity treatement 16
n_cols AFTER multicolinearity treatement 13
attemp no.:  0 3s&knn__uniform&BoP&SMOTE  acc:  0.8121019108280255  accuracy_train:  0.8884615384615384  f1:  0.2337662337662338  f1_train:  0.8935128518971847  spec:  0.42857142857142855  :auc  0.6340809361287179  cfm:  
 [[246  47]
 [ 12   9]] 
 cols 13 Index(['feature_20', 'feature_22', 'feature_34', 'feature_60', 'feature_104',
       'feature_122', 'feature_130', 'feature_131', 'feature_206',
       'feature_248', 'feature_349', 'feature_427', 'feature_511'],
      dtype='object') 

n_cols BEFORE multicolinearity treatement 16
n_cols AFTER multicolinearity treatement 14
attemp no.:  1 3s&knn__uniform&BoP&ROSE  acc:  0.7802547770700637  accuracy_train:  0.8401709401709402  f1:  0.24175824175824173  f1_train:  0.8398972602739726  spec:  0.5238095238095238  :auc  0.6612221680481066  cfm:  
 [[234  59]
 [ 10  11]] 
 cols 14 Index(['feature_1', 'feature_20', 'feature_22', 'feature_34', 'feature_60'

100%|██████████| 100/100 [01:22<00:00,  1.21it/s]


n_cols BEFORE multicolinearity treatement 19
n_cols AFTER multicolinearity treatement 14
attemp no.:  2 3s&knn__uniform&BoS__shap&SMOTE  acc:  0.8375796178343949  accuracy_train:  0.8730769230769231  f1:  0.3013698630136986  f1_train:  0.8771203971865951  spec:  0.5238095238095238  :auc  0.6919388915975947  cfm:  
 [[252  41]
 [ 10  11]] 
 cols 14 Index(['feature_442', 'feature_206', 'feature_435', 'feature_118',
       'feature_133', 'feature_461', 'feature_66', 'feature_22', 'feature_148',
       'feature_349', 'feature_34', 'feature_124', 'feature_332',
       'feature_60'],
      dtype='object') 



100%|██████████| 100/100 [01:18<00:00,  1.28it/s]


n_cols BEFORE multicolinearity treatement 15
n_cols AFTER multicolinearity treatement 10
attemp no.:  3 3s&knn__uniform&BoS__shap&ROSE  acc:  0.8089171974522293  accuracy_train:  0.8363247863247864  f1:  0.2857142857142857  f1_train:  0.8370905997447895  spec:  0.5714285714285714  :auc  0.6986835689907362  cfm:  
 [[242  51]
 [  9  12]] 
 cols 10 Index(['feature_164', 'feature_22', 'feature_148', 'feature_65', 'feature_206',
       'feature_349', 'feature_118', 'feature_34', 'feature_60',
       'feature_133'],
      dtype='object') 



...

100%|██████████| 100/100 [01:27<00:00,  1.14it/s]


n_cols BEFORE multicolinearity treatement 12
n_cols AFTER multicolinearity treatement 8
attemp no.:  11 3s&MICE&BoS__shap&ROSE  acc:  0.8089171974522293  accuracy_train:  0.8226495726495726  f1:  0.2682926829268293  f1_train:  0.8199566160520608  spec:  0.5238095238095238  :auc  0.6765805298228506  cfm:  
 [[243  50]
 [ 10  11]] 
 cols 8 Index(['feature_148', 'feature_65', 'feature_206', 'feature_349',
       'feature_435', 'feature_332', 'feature_60', 'feature_34'],
      dtype='object') 



100%|██████████| 100/100 [00:46<00:00,  2.14it/s]


n_cols BEFORE multicolinearity treatement 6
n_cols AFTER multicolinearity treatement 4
attemp no.:  12 3s&MICE&BoS__gini&SMOTE  acc:  0.767515923566879  accuracy_train:  0.7482905982905983  f1:  0.2150537634408602  f1_train:  0.7299403943145347  spec:  0.47619047619047616  :auc  0.6322931903136682  cfm:  
 [[231  62]
 [ 11  10]] 
 cols 4 Index(['feature_427', 'feature_442', 'feature_65', 'feature_3'], dtype='object') 



KeyboardInterrupt: 

# Problems:
1. We should take tentative features, ranking>1 (BurutaShap -> need to ressearch how)
2. implement removing columns with high corr after FS_options 
3. Try other models, because RF is overfitting and we havd very bad F1

# --------------------Appendix--------------------

In [ ]:
# #------------------------------------
# X_train = replace_outlier(X_train, how)
# replace_outlier_options = ['NaN', '3s']
# #--
# X_train = impute_null_with_knn(X_train, which_weights) #
# impute_null_options = ['knn__distance', 'knn__uniform', 'MICE']

# X_train = impute_null_with_mice(X_train)
# #--
# BorutaShap_FS(X_train, y_train, method_option)
# list_method=['shap','gini']

# RFE_FS(X_train, y_train, classify) 
# list_clf=['RF','SVM']

# BorutaPy_FS(X_train, y_train)

# FS_options = ['BoS__shap', 'BoS__gini', 'RFE__RF', 'RFE__SVM', 'BoP']
# #--
# sampling(X_train, y_train, sampler)
# sampling_options = ['SMOTE','ROSE','ADASYN','SMOTEENN']

# #--------------------------------------

In [9]:
# l = []
# a=1
# b=2
# c=3

# l.append([a,b,c])
# l.append([b,c,a])

# l



[[1, 2, 3], [2, 3, 1]]

In [ ]:
# list_null_impute = [knnimputation_distance, MICEimputation_distance]
# list_null_outlier = [outlier_knn,outlier_3s]
# list_feat_selection = ['Boruta_RF', 'Boruta_shap', 'RFE']
# Boruta = ['RF', 'XGB']
# Boruta_shap = ['RF', 'XGB', 'kNN']
# RFE = ['RF', 'SVC']

In [ ]:
# #TESTING
# df = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9], [1,1,1]]),
#                    columns=['a', 'b', 'c'])
# df

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9
3,1,1,1
